In [1]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask

In [2]:
tif_path  = r"C:\Users\dskcy\UEInfo\Stats\GDP\GDP_025d (2000-2100)\025d\GDP2020.tif"
output_path = "clipped_output.tif"

In [6]:
# Load the vector (GeoDataFrame)
gdf = gpd.read_file(r"C:\Users\dskcy\UEInfo\EmissionsInventory_SolidWaste\assets\INDIA_STATES.geojson")

In [7]:
# Reproject vector to match raster CRS (important!)
with rasterio.open(tif_path) as src:
    if gdf.crs != src.crs:
        gdf = gdf.to_crs(src.crs)

    # Convert geometry to GeoJSON-like mapping
    geoms = [feature.__geo_interface__ for feature in gdf.geometry]

    # Clip the raster
    clipped_image, clipped_transform = mask(dataset=src, shapes=geoms, crop=True)
    clipped_meta = src.meta.copy()


In [8]:
# Update metadata
clipped_meta.update({
    "driver": "GTiff",
    "height": clipped_image.shape[1],
    "width": clipped_image.shape[2],
    "transform": clipped_transform
})

# Save the clipped raster
with rasterio.open(output_path, "w", **clipped_meta) as dest:
    dest.write(clipped_image)